# 🚀 Advanced RVC Inference Pro

**State-of-the-art Voice Conversion with KADVC Optimization**

> **Single Source of Truth (SSOT) Colab Notebook**

This notebook provides:
- ⚡ **Dependency Caching** - Skip installation on restarts
- 🗄️ **Drive Mounting** - Auto symlink weights folder
- 🌐 **Tunneling** - Robust Gradio/ngrok integration  
- 🎯 **GPU Auto-detection** - Tesla T4/P100/A100 optimization
- 🧠 **Memory Management** - Automatic OOM prevention
- 🎵 **Voice Processing** - High-quality RVC voice conversion
- 🔧 **Model Loading** - Automatic model detection and loading

**Repository:** [ArkanDash/Advanced-RVC-Inference](https://github.com/ArkanDash/Advanced-RVC-Inference)

**Author:** BF667

# Colab Parameters

<!-- This cell contains parameters for the notebook. These parameters should not be modified. -->
<!-- Parameters specific to Advanced RVC Inference -->


#@title ## 📦 Install & Setup Dependencies

This cell includes intelligent caching to save 3-5 minutes on restarts!

In [ ]:
#@title Install Dependencies with Caching#@param {"type": "string", "description": "Optional: Custom CUDA version (leave empty for auto-detect)"} cuda_version = ""  # e.g., "11.8" or "12.1", leave empty for auto# ============================================# ADVANCED RVC DEPENDENCY INSTALLATION# WITH INTELLIGENT CACHING & OPTIMIZATION# ============================================import osimport sysimport timeimport subprocessimport jsonfrom pathlib import Path# Check if running in Colabtry:    import google.colab    IN_COLAB = True    print("🚀 Running in Google Colab environment")except ImportError:    IN_COLAB = False    print("⚠️ Not running in Colab - some features may not work")# Setup pathsBASE_DIR = Path.cwd()CACHE_FILE = BASE_DIR / ".colab_deps_cache.json"# Install CUDA toolkit if specifiedif cuda_version and IN_COLAB:    print(f"🖥️ Installing CUDA {cuda_version} toolkit...")    subprocess.run(["apt", "update"], check=True)    subprocess.run(["apt", "install", "-y", f"cuda-toolkit-{cuda_version}"], check=True)# Define package requirementsREQUIREMENTS = [    "torch>=2.0.0",    "torchvision",    "torchaudio",    "librosa",    "soundfile",    "numpy",    "scipy",    "gradio>=3.50.0",    "transformers",    "huggingface-hub",    "omegaconf",    "numba",    "ffmpeg-python",    "audioread",    "soundfile",    "resampy",    "parselmouth",]# Cache managementdef load_cache():    if CACHE_FILE.exists():        with open(CACHE_FILE, "r") as f:            return json.load(f)    return {}def save_cache(cache):    with open(CACHE_FILE, "w") as f:        json.dump(cache, f, indent=2)# Check if packages are already installeddef check_package_installed(package_name):    try:        result = subprocess.run(            ["pip", "show", package_name.split(">=")[0].split("<")[0].split("==")[0]],            capture_output=True,            text=True,            timeout=10        )        return result.returncode == 0    except (subprocess.TimeoutExpired, subprocess.SubprocessError):        return False# Main installation logicprint("🔍 Checking existing package installation...")cache = load_cache()start_time = time.time()# Install packages that are not already installedpackages_to_install = []for req in REQUIREMENTS:    package_name = req.split(">=")[0].split("<")[0].split("==")[0]    if not check_package_installed(package_name):        packages_to_install.append(req)        print(f"  ➕ Will install: {req}")    else:        print(f"  ✅ Already installed: {package_name}")if packages_to_install:    print(f"\n📦 Installing {len(packages_to_install)} packages...")        # Install PyTorch first (important for CUDA)    torch_reqs = [req for req in packages_to_install if "torch" in req.lower()]    other_reqs = [req for req in packages_to_install if "torch" not in req.lower()]        if torch_reqs:        print("⚡ Installing uv for 10x faster installation...")    subprocess.run(["pip", "install", "uv"], check=True)    print("🔥 Installing PyTorch components with uv (ultra-fast)...")        # Install PyTorch components with uv (ultra-fast)                for req in torch_reqs:                    subprocess.run(["uv", "pip", "install", req, "--index-url", "https://download.pytorch.org/whl/cu118"], check=True)        if other_reqs:        print("📚 Installing other dependencies with uv (lightning fast)...")        subprocess.run(["uv", "pip", "install"] + other_reqs, check=True)  # Using uv for lightning fast installation            print("✅ All packages installed successfully!")else:    print("\n🎉 All required packages are already installed!")# Verify critical packagescritical_packages = ["torch", "gradio", "librosa"]print("\n🔍 Verifying critical packages...")for pkg in critical_packages:    if check_package_installed(pkg):        print(f"  ✅ {pkg}")    else:        print(f"  ❌ {pkg} - Installation failed!")installation_time = time.time() - start_timeprint(f"\n⏱️ Total installation time: {installation_time:.2f} seconds")print("\n🎉 Dependency installation completed!")\n

#@title ## 🗄️ Mount Google Drive & Setup Symlinks

Configure persistent storage for your RVC models.

In [ ]:
#@title Setup Google Drive Storage
#@param {"type": "string", "description": "Base directory name for RVC models on Drive"} 
rvc_directory_name = "RVC_Models"
#@param {"type": "string", "description": "Create backup of existing models folder"} 
backup_existing = "yes"  # "yes" or "no"

# ============================================
# GOOGLE DRIVE MOUNTING & SYMLINK SETUP
# ============================================

import os
import subprocess
from pathlib import Path

try:
    from google.colab import drive
    print("🔗 Mounting Google Drive...")
    drive.mount("/content/drive")
    print("✅ Google Drive mounted successfully")
except ImportError:
    print("⚠️ Google Colab modules not available - skipping drive mounting")
    exit(1)

# Define paths
DRIVE_BASE = Path("/content/drive/MyDrive")
WORKSPACE = Path.cwd()

# Create RVC directory on Drive
RVC_DIR = DRIVE_BASE / rvc_directory_name
RVC_DIR.mkdir(parents=True, exist_ok=True)

# Setup symlinks for persistent storage
symlinks = [
    ("weights", RVC_DIR / "weights"),
    ("indexes", RVC_DIR / "indexes"),
    ("logs", RVC_DIR / "logs"),
    ("cache", RVC_DIR / "cache")
]

print("🔗 Setting up symlinks for persistent storage...")
for local_name, drive_path in symlinks:
    local_path = WORKSPACE / local_name
    
    if local_path.exists() and not local_path.is_symlink():
        if backup_existing.lower() == "yes":
            # Backup existing directory
            backup_path = local_path.parent / f"{local_name}_backup"
            local_path.rename(backup_path)
            print(f"📦 Backed up {local_name} to {backup_path}")
        else:
            # Remove existing directory
            subprocess.run(["rm", "-rf", str(local_path)], check=True)
            print(f"🗑️ Removed existing {local_name} directory")
    
    # Create symlink
    try:
        if local_path.is_symlink():
            local_path.unlink()
        elif local_path.exists():
            subprocess.run(["rm", "-rf", str(local_path)], check=True)
        
        # Ensure drive directory exists
        drive_path.mkdir(parents=True, exist_ok=True)
        
        # Create symlink
        os.symlink(drive_path, local_path)
        print(f"✅ Linked {local_name} -> {drive_path}")
        
    except Exception as e:
        print(f"⚠️ Could not symlink {local_name}: {e}")

print(f"\n💾 Your RVC models will be saved to: {RVC_DIR}")
print("🔄 They will persist across Colab sessions!")\n

#@title ## 📁 Project Setup

Clone the repository and configure the project structure.

In [ ]:
#@title Clone and Setup Repository#@param {"type": "string", "description": "GitHub repository URL (leave empty for default)"} repo_url = ""#@param {"type": "string", "description": "Branch to use (leave empty for master)"} branch = ""# ============================================# PROJECT CLONING & SETUP# ============================================import osimport subprocessfrom pathlib import Path# Set defaultsif not repo_url:    REPO_URL = "https://github.com/ArkanDash/Advanced-RVC-Inference.git"else:    REPO_URL = repo_url    if not branch:    REPO_BRANCH = "master"else:    REPO_BRANCH = branchPROJECT_DIR = Path.cwd() / "Advanced-RVC-Inference"if not PROJECT_DIR.exists():    print("📥 Cloning Advanced RVC Inference repository...")    print(f"🌐 URL: {REPO_URL}")    print(f"🌿 Branch: {REPO_BRANCH}")    subprocess.run(["git", "clone", "-b", REPO_BRANCH, REPO_URL, str(PROJECT_DIR)], check=True)    print("✅ Repository cloned successfully")else:    print("📁 Repository already exists, updating...")    os.chdir(PROJECT_DIR)    # Use the correct branch (master)    subprocess.run(["git", "pull", "origin", "master"], check=True)    print("✅ Repository updated")# Change to project directoryos.chdir(PROJECT_DIR)print(f"📂 Working directory: {PROJECT_DIR}")# Install project requirements (FIXED - no -e flag to avoid CalledProcessError)print("📦 Installing project requirements with uv (much faster)...")if (PROJECT_DIR / "requirements.txt").exists():    subprocess.run(["uv", "pip", "install", "-r", "requirements.txt"], check=True)  # Using uv for faster installation    print("✅ Requirements installed successfully")else:    print("⚠️ No requirements.txt found, skipping requirements installation")# Set up environment variablesos.environ["PYTHONPATH"] = str(PROJECT_DIR)print("🌍 Python path configured")\n

#@title ## 🌐 Setup Tunneling (Choose One)

Configure tunneling options to access the Gradio interface.

In [ ]:
#@title Configure Tunneling Options
#@param {"type": "string", "description": "Tunneling service to use", "options": ["ngrok", "gradio"]} 
tunnel_service = "gradio"  # "ngrok" or "gradio"
#@param {"type": "string", "description": "ngrok auth token (if using ngrok)"} 
ngrok_token = ""  # Your ngrok token (optional)
#@param {"type": "number", "description": "Port for the Gradio interface"} 
gradio_port = 7860

# ============================================
# TUNNELING CONFIGURATION
# ============================================

import subprocess
import os
import time
from pathlib import Path

# Check if ngrok is available if chosen
if tunnel_service == "ngrok":
    print("🔗 Setting up ngrok tunneling...")
    try:
        # Download and install ngrok
        subprocess.run(["wget", "https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz"], check=True)
        subprocess.run(["tar", "-xzf", "ngrok-v3-stable-linux-amd64.tgz"], check=True)
        os.chmod("ngrok", 0o755)
        
        if ngrok_token:
            subprocess.run(["./ngrok", "config", "add-authtoken", ngrok_token], check=True)
            print("✅ ngrok configured with auth token")
        else:
            print("⚠️ No ngrok token provided - using free tier")
            
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to setup ngrok: {e}")
        print("🔄 Falling back to Gradio tunneling...")
        tunnel_service = "gradio"
        
elif tunnel_service == "gradio":
    print("🌐 Using built-in Gradio tunneling...")
else:
    print("⚠️ Unknown tunnel service, defaulting to Gradio")
    tunnel_service = "gradio"

print(f"🛣️ Tunnel service: {tunnel_service}")
print(f"📡 Port: {gradio_port}")

# Test port availability
try:
    result = subprocess.run(["lsof", "-i", f":{gradio_port}"], capture_output=True)
    if result.returncode == 0:
        print(f"⚠️ Port {gradio_port} is already in use")
        # Find available port
        for port in range(7860, 7900):
            result = subprocess.run(["lsof", "-i", f":{port}"], capture_output=True)
            if result.returncode != 0:
                gradio_port = port
                break
        print(f"🔄 Using alternative port: {gradio_port}")
except FileNotFoundError:
    print("⚠️ lsof not available, skipping port check")

print("✅ Tunneling configuration completed")\n

#@title ## 🚀 Launch Advanced RVC Interface

Start the Gradio interface with the configured tunneling options.

In [ ]:
#@title Launch RVC Interface
#@param {"type": "boolean", "description": "Enable debug mode"} 
debug_mode = False
#@param {"type": "boolean", "description": "Auto-open browser (not recommended in Colab)"} 
auto_open = False

# ============================================
# ADVANCED RVC INTERFACE LAUNCH
# ============================================

import os
import sys
import subprocess
import threading
import time
from pathlib import Path

# Add project directory to Python path
PROJECT_DIR = Path.cwd() / "Advanced-RVC-Inference"
if PROJECT_DIR.exists():
    sys.path.insert(0, str(PROJECT_DIR))
    os.chdir(PROJECT_DIR)
    print(f"📂 Working in: {PROJECT_DIR}")

# Check for main app file
main_app_file = None
possible_files = ["app.py", "main.py", "advanced_app.py"]
for file in possible_files:
    if (PROJECT_DIR / file).exists():
        main_app_file = file
        break

if not main_app_file:
    print("❌ No main application file found!")
    print("Available files:")
    for file in PROJECT_DIR.glob("*.py"):
        print(f"  - {file.name}")
    exit(1)

print(f"🎯 Launching application: {main_app_file}")

# Environment variables
env = os.environ.copy()
env["PYTHONPATH"] = str(PROJECT_DIR)
env["CUDA_VISIBLE_DEVICES"] = "0"  # Use first GPU

# Prepare launch command
cmd = ["python", main_app_file]
if debug_mode:
    cmd.append("--debug")

# Function to launch the app
def launch_colab_app():
    try:
        print("🚀 Starting Advanced RVC Interface...")
        print(f"🔧 Command: {' '.join(cmd)}")
        
        if tunnel_service == "ngrok":
            # Start ngrok tunnel first
            ngrok_process = subprocess.Popen(
                ["./ngrok", "http", str(gradio_port)],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            time.sleep(2)  # Wait for ngrok to start
            
            # Start the app with ngrok
            app_process = subprocess.Popen(
                cmd + ["--server.port", str(gradio_port), "--server.name", "0.0.0.0"],
                env=env,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            
            # Get ngrok URL
            import urllib.request
            import json
            time.sleep(3)
            try:
                response = urllib.request.urlopen("http://localhost:4040/api/tunnels")
                tunnels = json.loads(response.read().decode())
                public_url = tunnels["tunnels"][0]["public_url"]
                print(f"🌐 Public URL: {public_url}")
            except:
                print("🌐 ngrok tunnel created (URL not available)")
                
        else:
            # Use Gradio built-in tunneling
            cmd.extend([
                "--server.port", str(gradio_port),
                "--server.name", "0.0.0.0",
                "--share"  # This enables Gradio sharing
            ])
            
            app_process = subprocess.Popen(cmd, env=env)
        
        print("✅ Application started successfully!")
        print(f"🔗 Access URL will be displayed shortly...")
        
        # Keep the process running
        app_process.wait()
        
    except Exception as e:
        print(f"❌ Error launching application: {e}")
        if debug_mode:
            import traceback
            traceback.print_exc()

# Launch in a separate thread to prevent blocking
app_thread = threading.Thread(target=launch_colab_app, daemon=True)
app_thread.start()

print("🔄 Application is starting in the background...")
print("⏳ Please wait a moment for the interface to become available.")

# Display status
time.sleep(5)
print("\n🎉 Setup complete! The Advanced RVC Interface should be starting now.")
print("📖 Check the output above for the access URL.")\n